In [9]:
import os

print(os.path.abspath(os.curdir))

e:\Data Kuliah\S2 ITS\Semester 5\Sentiment Analysis\drone-severity-multitask


In [3]:
import os
os.chdir("..")

In [4]:
print(os.path.abspath(os.curdir))

e:\Data Kuliah\S2 ITS\Semester 5\Sentiment Analysis\drone-severity-multitask


In [6]:
!pip install InstructorEmbedding


[notice] A new release of pip available: 22.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [13]:
"""
This is a simple application for sentence embeddings: clustering

Sentences are mapped to sentence embeddings and then agglomerative clustering with a threshold is applied.
"""
from sentence_transformers import SentenceTransformer
from sklearn.cluster import AgglomerativeClustering
import numpy as np
import pandas as pd
from InstructorEmbedding import INSTRUCTOR

instructor_embedding = INSTRUCTOR('hkunlp/instructor-base')
# instructor_embedding = INSTRUCTOR('hkunlp/instructor-large')
# instructor_embedding = INSTRUCTOR('hkunlp/instructor-xl')

embedder = SentenceTransformer('all-mpnet-base-v2')
embedder = SentenceTransformer('all-distilroberta-v1')
embedder = SentenceTransformer('all-MiniLM-L12-v2')
embedder = SentenceTransformer('all-MiniLM-L6-v2')
embedder = SentenceTransformer('multi-qa-mpnet-base-dot-v1')
# Try INSTRUCTOR
# query = [['Represent the Drone Log message for clustering: ', prompt]]
log_dict = []
# Corpus with example sentences
dataset = pd.read_csv('dataset/merged-manual-unique.csv')
for ind in dataset.index:
    log_dict.append(['Represent the Drone Log message for clustering: ', dataset['message'][ind]])
    
corpus = dataset['message'].to_list()
# corpus = ['A man is eating food.',
#           'A man is eating a piece of bread.',
#           'A man is eating pasta.',
#           'The girl is carrying a baby.',
#           'The baby is carried by the woman',
#           'A man is riding a horse.',
#           'A man is riding a white horse on an enclosed ground.',
#           'A monkey is playing drums.',
#           'Someone in a gorilla costume is playing a set of drums.',
#           'A cheetah is running behind its prey.',
#           'A cheetah chases prey on across a field.'
#           ]
corpus_embeddings = instructor_embedding.encode(log_dict)

# Normalize the embeddings to unit length
corpus_embeddings = corpus_embeddings /  np.linalg.norm(corpus_embeddings, axis=1, keepdims=True)

# Perform agglomerative clustering
clustering_model = AgglomerativeClustering(n_clusters=None, linkage='average', distance_threshold=0.05, metric='cosine') #, affinity='cosine', linkage='average', distance_threshold=0.4)
clustering_model.fit(corpus_embeddings)
cluster_assignment = clustering_model.labels_

clustered_sentences = {}
for sentence_id, cluster_id in enumerate(cluster_assignment):
    if cluster_id not in clustered_sentences:
        clustered_sentences[cluster_id] = []

    clustered_sentences[cluster_id].append(corpus[sentence_id])
print(len(clustered_sentences))
for i, cluster in clustered_sentences.items():
    print("Cluster ", i+1)
    print(cluster)
    print("")

load INSTRUCTOR_Transformer
max_seq_length  512
361
Cluster  144
['A passenger aircraft is approaching. Descend as soon as possible', 'Another aircraft is nearby. Another aircraft is approaching. Descend as soon as possible']

Cluster  19
['A passenger aircraft is nearby. Fly with caution', 'Another aircraft is nearby. Fly with caution', 'Fly with caution and ensure the aircraft remains within your line of sight.', 'Manned aircraft detected nearby. Fly with caution']

Cluster  259
['Abnormal compass function or GPS signal detected. Aircraft switched to ATTI mode']

Cluster  224
['Accelerator is Over Range']

Cluster  285
['Account not logged in. Flight altitude and distance restricted to nnn and nnn. Check and log in']

Cluster  8
['ActiveTrack Flight Ended: Aircraft is entering a restricted area', 'ActiveTrack Flight Stopped Aircraft Entering Restricted Area', 'Aircraft is entering a restricted area']

Cluster  356
['ActiveTrack Flight Ended: Cannot find subject']

Cluster  281
['Acti